In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import random
import math
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.models import load_model
from keras.utils import plot_model
import h5py
from ann_visualizer.visualize import ann_viz

loading dataframes

In [ ]:
# train scores when training with teacher 
with_teacher_train = pd.read_csv('with_teacher_train.csv')
with_teacher_train = with_teacher_train.drop('Unnamed: 0', axis = 1)
# train scores when training without teacher 
without_teacher_train = pd.read_csv('without_teacher_train.csv')
without_teacher_train = without_teacher_train.drop('Unnamed: 0', axis = 1)

In [ ]:
test_all = pd.read_csv('test_all.csv')
test_all = test_all.drop('Unnamed: 0', axis = 1)
test_teachers = pd.read_csv('test_teachers.csv')
test_teachers = test_teachers.drop('Unnamed: 0', axis = 1)

In [ ]:
###############################
## Analysis helper functions ##
###############################
from scipy import optimize
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

def test_func(x, a, b):
    return a * x + b 

def plot_train(y, color, xlabel, ylabel, withLine):
    plt.figure(figsize=(7.5,5))
    params, params_covariance = optimize.curve_fit(test_func, range(len(y)), y, p0=[2, 2])
    print(params)
    sns.lineplot(x=range(len(y)), y=y,  color = color)
    plt.gca().set_ylim(-3500, 100)
    plt.plot(range(len(y)), test_func(range(len(y)), params[0], params[1]), range(len(y)), y, color = color, linewidth = 3)
    plt.show()
    
def stats(y):
    X = range(len(y))
    y = y
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())
    
def line_plot_test(df):
    plt.figure(figsize=(10,5))
    sns.lineplot(data=df, x='game_num', y='test_score', hue='condition')
    plt.plot(range(len(y)), test_func(range(len(y)), params[0], params[1]), range(len(y)), y, color = color, linewidth = 3)
    plt.show()
    
def bar_plot_test(df, x, y, palette):
    plt.figure(figsize=(6,5))
#     plt.gca().set_ylim(-1800, 0)
    sns.barplot(data=df, x=x, y=y, palette=palette, errwidth=1, capsize=0.1)
    plt.show() # '#ef945f', 

### Training analysis

In [ ]:
plot_train(list(without_teacher_train['train_scores']), '#75a863', 'games', 'train scores without teacher', True)

In [ ]:
stats(list(without_teacher_train['train_scores'])) 
# with_teacher_train_grouped = with_teacher_train.groupby(np.arange(len(with_teacher_train))//5).median()

In [ ]:
plot_train(list(with_teacher_train['train_scores'])[:126], '#85d1f7', 'games', 'train scores with teacher', True)

In [ ]:
stats(list(with_teacher_train['train_scores'])[:126]) 

### Testing analysis

In [ ]:
bar_plot_test(test_all, 'condition', 'test_score', palette = ['#c1943f', '#69aeea', '#67cc63']) 

##### Randomized teacher performance across experience

In [ ]:
test_with_random = test_all[test_all['condition'] == 'with_random']
test_with_random['student'] =  [i//10 for i in range(50)]
bar_plot_test(test_with_random, 'student', 'test_score', palette = "OrRd")

In [ ]:
test_with_random_student_means = test_with_random.groupby(np.arange(len(test_with_random))//10).median()['test_score']
stats(test_with_random_student_means)

##### Without teacher performance across experience

In [ ]:
test_without_teacher = test_teachers[test_teachers['condition'] == 'without_teacher']
test_without_teacher['student'] = [i//25 for i in range(250)]
bar_plot_test(test_without_teacher, 'student', 'test_score', palette="YlGn")

In [ ]:
test_without_teacher_student_means = test_without_teacher.groupby(np.arange(len(test_without_teacher))//25).median()['test_score']
stats(test_without_teacher_student_means)

##### With RL teacher performance across experience

In [ ]:
test_with_teacher = test_teachers[test_teachers['condition'] == 'with_teacher']
test_with_teacher['student'] = [i//25 for i in range(250)]
bar_plot_test(test_with_teacher, 'student', 'test_score', palette="GnBu")

In [ ]:
test_with_teacher_student_means = test_with_teacher.groupby(np.arange(len(test_with_teacher))//25).median()['test_score']
stats(test_with_teacher_student_means)

In [ ]:
### number of rounds 
bar_plot_test(test_all, 'condition', 'num_rounds', palette = ['#b27357', '#69aeea', '#67cc63']) # just get the first half of without_teacher 

In [ ]:
# material differences 
bar_plot_test(test_all, 'condition', 'mat_diff', palette = ['#b27357', '#69aeea', '#67cc63']) # just get the first half of without_teacher 

In [ ]:
# proportion of optimal moves 
bar_plot_test(test_all, 'condition', 'match_prop', palette = ['#b27357', '#69aeea', '#67cc63']) # just get the first half of without_teacher 

## supplementary analysis

In [ ]:
### TEST SCORES 
plt.figure(figsize=(10,5))
plt.plot(range(len(without_teacher_test_scores)), without_teacher_test_scores, color = '#0be8b7', linewidth = 3)
plt.show()

In [ ]:
### MATCHES 
plt.figure(figsize=(10,5))
plt.plot(range(len(without_teacher_test_matches)), without_teacher_test_matches, color = '#0be8b7', linewidth = 3)
plt.show()

In [ ]:
### ROUNDS
plt.figure(figsize=(10,5))
plt.plot(range(len(without_teacher_test_rounds)), without_teacher_test_rounds, color = '#0be8b7', linewidth = 3)
plt.show()

In [ ]:
### MATERIAL DIFFERENCES
plt.figure(figsize=(10,5))
plt.plot(range(len(without_teacher_test_material)), without_teacher_test_material, color = '#0be8b7', linewidth = 3)
plt.show()